In [3]:
import collections
import json
import urllib2

In [4]:
simulation = dict(
    scenarios = [
        dict(
            test_case = dict(
                familles = [
                    dict(
                        parents = ['ind0', 'ind1'],
                        enfants = ['ind2', 'ind3'],
                        ),
                    ],
                foyers_fiscaux = [
                    dict(
                        declarants = ['ind0', 'ind1'],
                        personnes_a_charge = ['ind2', 'ind3'],
                        ),
                    ],
                individus = [
                    dict(
                        activite = u'Actif occupé',
                        birth = '1970-01-01',
                        id = 'ind0',
                        sali = 35000,
                        statmarit = u'Marié',
                        ),
                    dict(
                        activite = u'Actif occupé',
                        birth = '1970-01-02',
                        id = 'ind1',
                        sali = 35000,
                        statmarit = u'Marié',
                        ),
                    dict(
                        activite = u'Étudiant, élève',
                        birth = '2000-01-03',
                        id = 'ind2',
                        ),
                    dict(
                        activite = u'Étudiant, élève',
                        birth = '2000-01-04',
                        id = 'ind3',
                        ),
                    ],
                menages = [
                    dict(
                        personne_de_reference = 'ind0',
                        conjoint = 'ind1',
                        enfants = ['ind2', 'ind3'],
                        ),
                    ],
                ),
            year = 2013,
            ),
        ],
    )

request = urllib2.Request('http://api.openfisca.fr/api/1/simulate', headers = {
    'Content-Type': 'application/json',
    'User-Agent': 'OpenFisca-Notebook',
    })
try:
    response = urllib2.urlopen(request, json.dumps(simulation))
except urllib2.HTTPError as response:
    print response.read()
    raise
response_dict = json.loads(response.read(), object_pairs_hook = collections.OrderedDict)
print json.dumps(response_dict, indent = 2)
tree = response_dict['value']

{
  "apiVersion": "1.0", 
  "method": "/api/1/simulate", 
  "params": {
    "scenarios": [
      {
        "test_case": {
          "foyers_fiscaux": [
            {
              "personnes_a_charge": [
                "ind2", 
                "ind3"
              ], 
              "declarants": [
                "ind0", 
                "ind1"
              ]
            }
          ], 
          "individus": [
            {
              "sali": 35000, 
              "activite": "Actif occup\u00e9", 
              "statmarit": "Mari\u00e9", 
              "birth": "1970-01-01"
            }, 
            {
              "sali": 35000, 
              "activite": "Actif occup\u00e9", 
              "statmarit": "Mari\u00e9", 
              "birth": "1970-01-02"
            }, 
            {
              "activite": "\u00c9tudiant, \u00e9l\u00e8ve", 
              "birth": "2000-01-03"
            }, 
            {
              "activite": "\u00c9tudiant, \u00e9l\u00e8ve", 
         

In [5]:
def any(iterator):
    for item in iterator:
        if item:
            return True
    return False

def iter_columns_from_tree(node, base_values = None):
    children = node.get('children')
    if children:
        child_base_values = base_values
        for child in children:
            for column in iter_columns_from_tree(child, base_values = child_base_values):
                yield column
            if child_base_values is None:
                child_base_values = child['values']
            else:
                child_base_values = [
                    base_value + value
                    for base_value, value in zip(child_base_values, child['values'])
                    ]

    if any(value != 0 for value in node['values']):
        column = dict(
            base_values = base_values or [0] * len(node['values']),
            code = node['code'],
            )
        column.update(node)
        yield column

In [6]:
for column_index, column in enumerate(iter_columns_from_tree(tree)):
    print u'{:40s} | {}'.format(column['name'], ' | '.join('{:6.0f}'.format(value) for value in column['values']))

Salaires super bruts                     | 124209
Cotisations sociales patronales          | -38233
Salaires bruts                           |  85976
Cotisations sociales salariées           | -11668
CSG déductible (salaires)                |  -4308
Salaires imposables                      |  70000
CSG non déductible (salaires)            |  -2027
CRDS (salaires)                          |   -422
Salaires nets                            |  67550
Revenu du travail                        |  67550
Revenus nets                             |  67550
Allocations familiales                   |   1532
CRDS (prestations familiales)            |     -8
Prestations familiales                   |   1524
Prestations sociales                     |   1524
Impôt sur le revenu                      |  -4770
Impôts directs                           |  -4770
Revenu disponible                        |  64304


In [7]:
from __future__ import division

from matplotlib.patches import FancyArrow

bar_width = 0.8
title = u'Revenu annuel'
value_index = 0
xlabel = u'Prélèvements et prestations sociales'
ylabel = u'Montant en €'

figure = pyplot.figure(figsize = (15, 9), dpi = 100)
ax = figure.add_subplot(111)
ax.set_title(title)
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.set_xscale('linear')
ax.set_yscale('linear')
#ax.set_xlim(*xlim)
#ax.set_ylim(*ylim)
#ax.hold(False)

names = []
for column_index, column in enumerate(iter_columns_from_tree(tree)):
    r, g, b = column['color']
    base_value = column['base_values'][0]
    value = column['values'][0]
    arrow = FancyArrow(column_index + bar_width / 2, base_value , 0, value, width = bar_width,
        fc = (r / 255, g / 255, b / 255), linewidth = 0.5, edgecolor = 'black', label = column['name'],
        picker = True, length_includes_head = True, head_width = bar_width, head_length = abs(value / 15))
    ax.add_patch(arrow)
    rounded_value = round(value)
    ax.text(column_index + bar_width / 2, max(base_value, value) + 1, str(int(rounded_value)), horizontalalignment = 'center',
        verticalalignment = 'bottom', color = 'black' if rounded_value >= 0 else 'red', weight = 'bold')
    names.append(column['short_name'])

xlim = (-bar_width / 2, len(names) - 1 + bar_width * 1.5)
#ax.hold(True)
ax.plot(xlim, [0, 0], color = 'black')
ax.set_xticklabels(names, rotation = '45')
ax.set_xticks(np.arange(len(names)) + bar_width / 2)
ax.set_xlim(xlim)
#ticks = ax.get_xticklines()
#for tick in ticks:
#    tick.set_visible(False)

#m, M = ax.get_ylim()
#ax.set_ylim((m, 1.05*M))


(-0.4, 18.2)